In [2]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd

In [3]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance/60)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance/60))

In [4]:
def create_data_model():
    matrix = pd.read_csv('distance.csv')
    m_list = []
    
    for i in range(len(matrix)):
        temp = []
        for j in range(len(matrix)):
            temp.append(matrix.iloc[i,j])
        m_list.append(temp)

    data = {}
    data['distance_matrix'] = m_list
    data['num_vehicles'] = 8
    data['depot'] = 35
    return data

In [5]:
# Instantiate the data problem.
data = create_data_model()

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

In [6]:
def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node] 

In [7]:
transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [8]:
dimension_name = 'Distance'
routing.AddDimension(
    transit_callback_index,
    100000,  # no slack
    300000,  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

In [9]:
def print_solution_only_num():
    output = []
    for vehicle_id in range(data['num_vehicles']):
        Output = []
        index = routing.Start(vehicle_id)
        while not routing.IsEnd(index):
            Output.append(manager.IndexToNode(index))
            index = solution.Value(routing.NextVar(index))
        Output.append(manager.IndexToNode(index))  
        output.append(Output)
    return output

In [10]:
solution = routing.SolveWithParameters(search_parameters)

In [11]:
if solution:
    print_solution(data, manager, routing, solution)
    solution = print_solution_only_num()
else:
    print('No solution found !')

Objective: 167288
Route for vehicle 0:
 35 ->  19 ->  24 ->  6 ->  12 ->  17 -> 35
Distance of the route: 16.0m

Route for vehicle 1:
 35 ->  10 ->  16 ->  7 ->  26 -> 35
Distance of the route: 25.5m

Route for vehicle 2:
 35 ->  1 ->  9 ->  20 ->  27 ->  3 -> 35
Distance of the route: 21.266666666666666m

Route for vehicle 3:
 35 ->  15 ->  25 ->  22 ->  2 -> 35
Distance of the route: 26.0m

Route for vehicle 4:
 35 ->  4 ->  32 ->  30 ->  31 ->  0 ->  28 -> 35
Distance of the route: 22.9m

Route for vehicle 5:
 35 ->  29 ->  21 ->  5 ->  33 ->  18 -> 35
Distance of the route: 25.866666666666667m

Route for vehicle 6:
 35 ->  8 ->  34 ->  23 -> 35
Distance of the route: 25.966666666666665m

Route for vehicle 7:
 35 ->  11 ->  13 ->  14 -> 35
Distance of the route: 24.633333333333333m

Maximum of the route distances: 26.0m


In [12]:
import folium

In [13]:
cluster = pd.read_csv('clustered.csv')

In [14]:
cluster

,lat,lng,clust_s
0,35.212616,126.842246,24.462366
1,35.208391,126.864639,20.000000
2,35.219626,126.841940,10.000000
3,35.216086,126.849209,3.000000
4,35.199945,126.829959,28.000000
5,35.213612,126.875127,6.000000
6,35.212114,126.834243,18.000000
7,35.221488,126.852125,14.000000
8,35.189908,126.813245,5.000000
9,35.206824,126.860538,1.000000


In [15]:
mean_x = cluster['lng'].mean()
mean_y = cluster['lat'].mean()

In [16]:
m = folium.Map( location = [mean_y,mean_x],zoom_start=14)
for i in range(len(cluster)):
    folium.Marker(
        location = [cluster.iloc[i]['lng'],cluster.iloc[i]['lat']]
    ).add_to(m)
    
m

In [23]:
color = ['green','red','blue','yellow','pink','pupple','black','orange','blue']
num = 0
for i in solution:
    
    for j in range(1,len(i)-1):
        folium.PolyLine(locations=[[cluster.iloc[i[j]-1]['lat'],cluster.iloc[i[j]-1]['lng']], [cluster.iloc[i[j+1]-1]['lat'],cluster.iloc[i[j+1]-1]['lng']]]
                        ,tooltip='Polyline'
                        ,color = color[num]
                       ).add_to(m)
    num+=1

In [24]:
m